In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as sts
import statsmodels.stats as stm
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

In [2]:
permcrop5_df = pd.read_csv('../CSVs/Permanent_Crop_NUTS5_2019.csv', sep=",")
tempcrop5_df = pd.read_csv('../CSVs/Temporary_Crop_NUTS5_2019.csv', sep=",")
education5_df = pd.read_csv('../CSVs/Education_NUTS5_2019.csv', sep=",")
labour5_df = pd.read_csv('../CSVs/Labour_NUTS5_2019.csv', sep=",")
prod5_df = pd.read_csv('../CSVs/Production_NUTS5_2019.csv', sep=",")

In [3]:
permcrop5_df.head()

,NutsID,region_name,NUTS_level,pc.year,pc.area,pcn.crop_name
0,111160101,Aboim das Choças,5,2019,13,Total
1,111160101,Aboim das Choças,5,2019,0,Fresh fruit plantations (excluding citrus plan...
2,111160101,Aboim das Choças,5,2019,0,Citrus plantations
3,111160101,Aboim das Choças,5,2019,0,Fruit plantations (subtropical climate zones)
4,111160101,Aboim das Choças,5,2019,0,Nuts plantations


In [4]:
prod5_df.head()

,NutsID,region_name,NUTS_level,p.year,p.value_eur,p.area_ha
0,111160101,Aboim das Choças,5,2019,101072,1921.9
1,111160102,Aguiã,5,2019,156561,1642.8
2,111160104,Ázere,5,2019,140132,2428.6
3,111160105,Cabana Maior,5,2019,189450,152.7
4,111160106,Cabreiro,5,2019,612235,221.4


In [5]:
all_dfs = [permcrop5_df, tempcrop5_df, education5_df, labour5_df, prod5_df]

def map_value(value):
    digits = str(value)[:2]
    if digits == '11':
        return 'Norte'
    elif digits == '15':
        return 'Algarve'
    elif digits == '16':
        return 'Centro'
    elif digits == '17':
        return 'Lisboa'
    elif digits == '18':
        return 'Alentejo'
    elif digits == '20':
        return 'Açores'
    elif digits == '30':
        return 'Madeira'
    else:
        return '-'

for df in all_dfs:
    df['NUTS2_region'] = df['NutsID'].apply(map_value)
    print(df.head())

      NutsID       region_name  NUTS_level  pc.year  pc.area  \
0  111160101  Aboim das Choças           5     2019       13   
1  111160101  Aboim das Choças           5     2019        0   
2  111160101  Aboim das Choças           5     2019        0   
3  111160101  Aboim das Choças           5     2019        0   
4  111160101  Aboim das Choças           5     2019        0   

                                       pcn.crop_name NUTS2_region  
0                                              Total        Norte  
1  Fresh fruit plantations (excluding citrus plan...        Norte  
2                                 Citrus plantations        Norte  
3      Fruit plantations (subtropical climate zones)        Norte  
4                                   Nuts plantations        Norte  
      NutsID       region_name  NUTS_level  tc.year  tc.area  tc.hold  \
0  111160101  Aboim das Choças           5     2019       12       20   
1  111160101  Aboim das Choças           5     2019        6 

In [7]:
prod5_df['NUTS2_region'].unique()

array(['Norte', 'Algarve', 'Centro', 'Lisboa', 'Alentejo', 'Açores',
       'Madeira'], dtype=object)

# Education

In [ ]:
education5_df.head()

In [ ]:
education5_df['el.education_level'].fillna('None', inplace=True)

In [ ]:
ed_no_totals = education5_df[education5_df['el.education_level'] != 'Total']
ed_totals = education5_df[education5_df['el.education_level'] == 'Total']

In [ ]:
ed_no_totals['total_count'] = ed_no_totals.groupby('region_name')['el.education_level'].transform('count')

In [ ]:
ed_totals.head()